<a href="https://colab.research.google.com/github/ms624atyale/PictureBook_Project_Demo/blob/main/3_mattrbyauthor_ccw_wordcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐹🐾 <font color = 'green'>>**Python code for MATTR broken into two code cells for tableau used in publishing paper**</font> Including ['Data Size] using len( )

In [ ]:
!pip install pandas
!pip install lexical_diversity
import pandas as pd #Import Pandas Package
import lexical_diversity as ld

#!git clone https://github.com/ms624atyale/PictureBook_Project_Demo
%cd /content/PictureBook_Project_Demo/Data_Frame

import glob
# Load all .txt files
fns = glob.glob('*.txt')

# List to hold each temporary DataFrame
df_list = []

# Load each file and append to list
for fn in fns:
    dftmp = pd.read_csv(fn, sep='@')
    df_list.append(dftmp)

# Concatenate all and reset index
df = pd.concat(df_list, ignore_index=True)

%cd /content

df.to_csv("./pg_2creators_10books.csv")

df
print(df)
print('\n')





import pandas as pd
from lexical_diversity import lex_div as ld

# Load the CSV (skip if already loaded)
df = pd.read_csv("./pg_2creators_10books.csv", sep=",")

#INSERTED BETWEEN START
!pip install textstat
import textstat

df['N_Sents'] = df['text'].apply(textstat.sentence_count)
df.to_csv('LD_result_with_Nsents.csv')


# Added column: String length
length = []

for i in range(0, len(df['text'])):
  LEN = len(df['text'][i])
  length.append(LEN)

df['Data size'] = length
print(df)



# Added column:  Splitted words, Length of splitted words
tsplit = []
splen = []

for i in range(0, len(df['text'])):
  TSP = df['text'][i].split()
  SPLEN = len(TSP)
  tsplit.append(TSP)
  splen.append(SPLEN)
  # print(TSP)

df['Splits'] = tsplit
df['N_Splits'] = splen
print(df)
#INSERTED BETWEEN END



# Added column: String length
#df['Lemma'] is a column from a pandas DataFrame, so len(df['Lemma']) gives you the number of rows (or entries) in that column.
lemma = []

for i in range(0, len(df['text'])):
  LEM = ld.flemmatize(df['text'][i])
  print(LEM)
  lemma.append(LEM)

df['Lemma'] = lemma

print('\n')
print(df['Lemma'])



# ADD LD indices

#1. Create empty lists.
TTR = []
MATTR = []

# 2. Getting LD index values for each cell:

for i in range(0, len(df['Lemma'])):
  flt = df['Lemma'][i]
  ttr = ld.ttr(flt)
  mattr = ld.mattr(flt)


  # Add values to each list
  TTR.append(ttr)
  MATTR.append(mattr)


# Add columns
df['TTR'] = TTR
df['MATTR'] = MATTR

# INSERTED BETWEEN START
df_selected = df[['creator','N_Splits','N_Sents','MATTR']]
df_selected
# INSERTED BETWEEN END



In [ ]:
# Clean up creator names
df['creator'] = df['creator'].str.strip().str.lower()

df_creator_ld = df.groupby("creator")['MATTR'].mean().reset_index()
df_creator_ld = df_creator_ld.round(6)

# --- STEP 6: Save and display results ---
df_creator_ld.to_csv('LD_by_creator.csv', index=False)
print("✅ Lexical diversity by creator saved as 'LD_by_creator.csv'!\n")
df_creator_ld

#<font color = 'red'> **Common Core Words for all 2 creators saved in a single .csv file**

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Load the dataset
file_path = "/content/pg_2creators_10books.csv"  # Change to your actual file name/content/pg_5authors_16books.csv
df_ccw = pd.read_csv(file_path)
# Ensure the text column exists
if 'text' not in df_ccw.columns or 'creator' not in df_ccw.columns:
    raise ValueError("CSV file must contain 'text' and 'creator' columns.")

# Combine all text data into one large string
all_text = " ".join(df_ccw['text'].astype(str))

# Tokenize words
words = word_tokenize(all_text.lower())  # Convert to lowercase

# Remove punctuation and stopwords
stop_words = set(stopwords.words('english'))
words = [word for word in words if word.isalpha() and word not in stop_words]  # Keep only words, no numbers/symbols

# Count word frequencies
word_counts = Counter(words)

# Convert to a DataFrame
common_words_df = pd.DataFrame(word_counts.most_common(100), columns=['Word', 'Frequency'])  # Top 100 words

# Save to CSV
common_words_df.to_csv("2creators_common_core_words.csv", index=False)

print("Common core words extracted and saved as '2creators_common_core_words.csv'!")

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Path to the combined CSV file (not a folder)
input_file = "/content/2creators_common_core_words.csv"

# Read the CSV containing word-frequency pairs
df = pd.read_csv(input_file)

# Convert to dictionary: {'word1': freq1, 'word2': freq2, ...}
word_freq = dict(zip(df['Word'], df['Frequency']))

# Create and generate the word cloud
wc = WordCloud(width=1000, height=500, background_color='white', colormap='tab10')
wc.generate_from_frequencies(word_freq)

# Display the word cloud inline in Colab
plt.figure(figsize=(15, 7))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud from All 2 Creators", fontsize=20)
plt.show()